##### This notebook preprocesses the initial training set and creates a cleaned version of the training set to be provided to the clustering algorithm.
In more detail it contains the following steps:
1. It encodes features (e.g., with one-hot encoding and binary-encoding) that contain series as values
2. It explores the encoded dataframe to remove unnecessary rows and columns
3. It runs dataprep and identifies basic preprocessing
4. It implements basic preprocessing actions on the corresponding features
5. It implements preprocessing on the different features' granularity
6. It identifies and reports the preprocessing actions need per each feature individually
7. It drops features that exceed a specific NaN threshold
8. It drops users that exceed a specific NaN threshold
9. It implements the preprocessing actions identified on step 2


In [5]:
import warnings
import datetime
import numpy as np
import pandas as pd
from dataprep.eda import create_report
from functions import training_set_preprocessing

warnings.filterwarnings("ignore")

Load the initial training set

In [6]:
training_df = pd.read_pickle('../data/training_df.pkl')
training_df

,id,date,hour,sleep_points,exertion_points,altitude,badge_type,badge_value,calories,distance,...,minutes_after_wakeup,time_in_bed,sleep_efficiency,main_sleep,consciousness_raising_category,counterconditioning_category,helping_relationships_category,stimulus_control_category,step_goal,place
0,621e2ff067b776a2403eb737,2021-12-22,19,NaN,NaN,10.0,NaN,NaN,27.35,19650.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,NaN,NaN,NaN,2.35,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,20.0,NaN,NaN,44.50,26880.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,20.0,NaN,NaN,46.06,32050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,NaN,NaN,NaN,2.35,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165086,621e36f967b776a240e5e7c9,2021-05-20,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WORK/SCHOOL
165087,621e36f967b776a240e5e7c9,2021-05-20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME
165088,621e36f967b776a240e5e7c9,2021-05-21,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRANSIT
165089,621e36f967b776a240e5e7c9,2021-05-21,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME


##### 1. Encode features that contain series as values

Find the features that contain series and arrays as values

In [7]:
features = list(training_df.columns)
del features[0:3]
preprocessing_features = []
for feature in features:
    if training_df[feature].apply(lambda d: True if isinstance(d, np.ndarray) or isinstance(d, pd.Series) else False).any():
        preprocessing_features.append(feature)
print(len(preprocessing_features))
preprocessing_features

19


['badge_type',
 'badge_value',
 'exercise',
 'exercise_calories',
 'exercise_avg_hr',
 'exercise_duration',
 'exercise_steps',
 'exercise_sedentary_minutes',
 'exercise_lightly_minutes',
 'exercise_fairly_minutes',
 'exercise_very_minutes',
 'exercise_Out of Range_zone_minutes',
 'exercise_Out of Range_zone_calories',
 'exercise_Fat Burn_zone_minutes',
 'exercise_Fat Burn_zone_calories',
 'exercise_Cardio_zone_minutes',
 'exercise_Cardio_zone_calories',
 'exercise_Peak_zone_minutes',
 'exercise_Peak_zone_calories']

Preprocessing for badge_type and badge_value

In [8]:
# encode a pair of categorical-numerical features
df_badges = training_df[['badge_type', 'badge_value']]
new_df = pd.concat([training_set_preprocessing.encode_row(row, list(df_badges.columns)) for _, row in df_badges.iterrows()], axis=1).T

# merge with the final dataframe and drop unnecessary columns
training_df = pd.concat([training_df, new_df], axis=1)
training_df.drop(columns=['badge_type', 'badge_value'], inplace=True)
training_df

,id,date,hour,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,...,helping_relationships_category,stimulus_control_category,step_goal,place,LIFETIME_DISTANCE,LIFETIME_FLOORS,DAILY_STEPS,DAILY_FLOORS,LIFETIME_WEIGHT_GOAL_SETUP,GOAL_BASED_WEIGHT_LOSS
0,621e2ff067b776a2403eb737,2021-12-22,19,NaN,NaN,10.0,27.35,19650.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,NaN,2.35,0.0,314.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,20.0,44.50,26880.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,20.0,46.06,32050.0,272.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,NaN,2.35,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165086,621e36f967b776a240e5e7c9,2021-05-20,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,WORK/SCHOOL,NaN,NaN,NaN,NaN,NaN,NaN
165087,621e36f967b776a240e5e7c9,2021-05-20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,HOME,NaN,NaN,NaN,NaN,NaN,NaN
165088,621e36f967b776a240e5e7c9,2021-05-21,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN
165089,621e36f967b776a240e5e7c9,2021-05-21,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,HOME,NaN,NaN,NaN,NaN,NaN,NaN


Preprocessing for exercise features (17)

In [9]:
# calculate the sum and avg of specific features
exercise_df = training_df.loc[:, 'exercise':'exercise_Peak_zone_calories']
sum_features = list(exercise_df.loc[:, 'exercise_steps': 'exercise_Peak_zone_calories'].columns)
sum_features.append('exercise_calories')
for feature in sum_features:
    exercise_df[feature] = exercise_df[feature].apply(lambda d: training_set_preprocessing.replace_with_sum(d))
exercise_df['exercise_avg_hr'] = exercise_df['exercise_avg_hr'].apply(lambda d: training_set_preprocessing.replace_with_avg(d))

# encode a pair of categorical-numerical features
encoding_df = exercise_df[['exercise', 'exercise_duration']]
new_df = pd.concat([training_set_preprocessing.encode_row(row, list(encoding_df.columns)) for _, row in encoding_df.iterrows()], axis=1).T

# merge with the final dataframe and drop unnecessary columns
exercise_df = pd.concat([exercise_df, new_df], axis=1)
exercise_df.drop(columns=['exercise', 'exercise_duration'], inplace=True)
delete_features = list(training_df.loc[:, 'exercise':'exercise_Peak_zone_calories'].columns)
training_df.drop(columns=delete_features, inplace=True)
training_df = pd.concat([training_df, exercise_df], axis=1)
training_df

,id,date,hour,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,...,Swim,Aerobic Workout,Tennis,Sport,Interval Workout,Spinning,Weights,Elliptical,Treadmill,Bootcamp
0,621e2ff067b776a2403eb737,2021-12-22,19,NaN,NaN,10.0,27.35,19650.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,NaN,2.35,0.0,314.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,20.0,44.50,26880.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,20.0,46.06,32050.0,272.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,NaN,2.35,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165086,621e36f967b776a240e5e7c9,2021-05-20,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165087,621e36f967b776a240e5e7c9,2021-05-20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165088,621e36f967b776a240e5e7c9,2021-05-21,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165089,621e36f967b776a240e5e7c9,2021-05-21,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
training_df.to_pickle('../data/encoded_training_df.pkl')

##### 2. Explore the encoded dataframe to remove unnecessary rows and columns
- Does the encoded dataframe contains duplicates?
- How many users do they participate in the training process based on the encoded dataframe?
- Does the encoded dataframe contains rows only with NaNs?
- Does the encoded dataframe contains columns only with NaNs?
- How many records derived out of official experiment days and do they affect this work?
- Concatenate the date and hour columns into one datetime object column.

In [41]:
training_df = pd.read_pickle('../data/encoded_training_df.pkl')
training_df

,id,date,hour,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,...,Swim,Aerobic Workout,Tennis,Sport,Interval Workout,Spinning,Weights,Elliptical,Treadmill,Bootcamp
0,621e2ff067b776a2403eb737,2021-12-22,19,NaN,NaN,10.0,27.35,19650.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,NaN,2.35,0.0,314.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,20.0,44.50,26880.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,20.0,46.06,32050.0,272.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,NaN,2.35,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165086,621e36f967b776a240e5e7c9,2021-05-20,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165087,621e36f967b776a240e5e7c9,2021-05-20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165088,621e36f967b776a240e5e7c9,2021-05-21,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165089,621e36f967b776a240e5e7c9,2021-05-21,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# check for duplicates based on all features
# Finding: 1 exact duplicate has been found - when all features were present there were not duplicates, but after features splitting this one duplicate has appeared.
# Approach: Drop them
training_df.drop_duplicates(inplace=True)
training_df

,id,date,hour,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,...,Swim,Aerobic Workout,Tennis,Sport,Interval Workout,Spinning,Weights,Elliptical,Treadmill,Bootcamp
0,621e2ff067b776a2403eb737,2021-12-22,19,NaN,NaN,10.0,27.35,19650.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,NaN,2.35,0.0,314.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,20.0,44.50,26880.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,20.0,46.06,32050.0,272.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,NaN,2.35,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165086,621e36f967b776a240e5e7c9,2021-05-20,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165087,621e36f967b776a240e5e7c9,2021-05-20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165088,621e36f967b776a240e5e7c9,2021-05-21,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165089,621e36f967b776a240e5e7c9,2021-05-21,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# find the users that are participating in the training
print(training_df['id'].nunique(), "users are participating in the training process.")

71 users are participating in the training process.


In [44]:
# find rows with no information - 5164 such rows have been found
columns = list(training_df.loc[:, 'sleep_points':].columns)
cleaned_training_df = training_df[~training_df[columns].isnull().all(1)]
cleaned_training_df

,id,date,hour,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,...,Swim,Aerobic Workout,Tennis,Sport,Interval Workout,Spinning,Weights,Elliptical,Treadmill,Bootcamp
0,621e2ff067b776a2403eb737,2021-12-22,19,NaN,NaN,10.0,27.35,19650.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,NaN,2.35,0.0,314.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,20.0,44.50,26880.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,20.0,46.06,32050.0,272.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,NaN,2.35,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165086,621e36f967b776a240e5e7c9,2021-05-20,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165087,621e36f967b776a240e5e7c9,2021-05-20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165088,621e36f967b776a240e5e7c9,2021-05-21,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165089,621e36f967b776a240e5e7c9,2021-05-21,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# find columns with no information - as expected 0 such columns have been found
for col in columns:
    if cleaned_training_df[col].isnull().all():
        cleaned_training_df.drop(columns=col, inplace=True)
cleaned_training_df

,id,date,hour,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,...,Swim,Aerobic Workout,Tennis,Sport,Interval Workout,Spinning,Weights,Elliptical,Treadmill,Bootcamp
0,621e2ff067b776a2403eb737,2021-12-22,19,NaN,NaN,10.0,27.35,19650.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,NaN,2.35,0.0,314.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,20.0,44.50,26880.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,20.0,46.06,32050.0,272.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,NaN,2.35,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165086,621e36f967b776a240e5e7c9,2021-05-20,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165087,621e36f967b776a240e5e7c9,2021-05-20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165088,621e36f967b776a240e5e7c9,2021-05-21,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165089,621e36f967b776a240e5e7c9,2021-05-21,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# find how many records derived out of the official experiment dates and if affect this work
cleaned_training_df = cleaned_training_df.sort_values(by='date', ascending=True)
cleaned_training_df['date'] = pd.to_datetime(cleaned_training_df['date'].astype("str"), format='%Y-%m-%d')
experiments_df = cleaned_training_df.loc[((cleaned_training_df['date'] > '2021-05-23') & (cleaned_training_df['date'] < '2021-07-27')) |  ((cleaned_training_df['date'] > '2021-11-14') & (cleaned_training_df['date'] < '2022-01-18'))]
experiments_df.reset_index(inplace=True, drop=True)
extra = pd.concat([cleaned_training_df,experiments_df]).drop_duplicates(keep=False)
print("There are", len(extra), "rows out of experiment dates from", extra['id'].nunique(), "unique users with 'normal distribution'.")

There are 45225 rows out of experiment dates from 69 unique users with 'normal distribution'.


In [48]:
# concatenate the date and hour columns into one datetime object column
cleaned_training_df['date'] = pd.to_datetime(cleaned_training_df['date']) + cleaned_training_df['hour'] * pd.to_timedelta('1H')
cleaned_training_df.drop(columns=['hour'], inplace=True)
cleaned_training_df

,id,date,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,moderately_active_minutes,...,Swim,Aerobic Workout,Tennis,Sport,Interval Workout,Spinning,Weights,Elliptical,Treadmill,Bootcamp
164878,621e375b67b776a240290cdc,2021-01-06 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164960,621e2f6167b776a240e082a9,2021-01-06 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164856,621e301367b776a24057738e,2021-01-06 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164874,621e362467b776a2404ad513,2021-01-08 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164858,621e30c867b776a240d4aa6c,2021-01-08 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56523,621e301e67b776a240608a72,2022-01-22 00:00:00,NaN,NaN,NaN,0.9,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164964,621e310d67b776a24003096d,2022-01-24 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164870,621e346f67b776a24081744f,2022-01-24 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164855,621e300767b776a2404dc717,2022-01-24 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
cleaned_training_df.to_pickle('../data/cleaned_encoded_training_df.pkl')

##### 3. Run dataprep and identify basic preprocessing

In [57]:
training_df = pd.read_pickle('../data/cleaned_encoded_training_df.pkl')
training_df

,id,date,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,moderately_active_minutes,...,Swim,Aerobic Workout,Tennis,Sport,Interval Workout,Spinning,Weights,Elliptical,Treadmill,Bootcamp
164878,621e375b67b776a240290cdc,2021-01-06 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164960,621e2f6167b776a240e082a9,2021-01-06 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164856,621e301367b776a24057738e,2021-01-06 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164874,621e362467b776a2404ad513,2021-01-08 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164858,621e30c867b776a240d4aa6c,2021-01-08 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56523,621e301e67b776a240608a72,2022-01-22 00:00:00,NaN,NaN,NaN,0.9,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164964,621e310d67b776a24003096d,2022-01-24 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164870,621e346f67b776a24081744f,2022-01-24 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164855,621e300767b776a2404dc717,2022-01-24 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
report = create_report(training_df, title='Training Dataframe')
report.show_browser()

  0%|          | 0/17571 [00:00<?, ?it/s]

##### 4. Implement basic preprocessing

step_goal preprocessing

In [58]:
# convert from categorical to numerical
training_df['step_goal'].replace(to_replace=['NO_GOAL'], value=[0], inplace=True)
training_df['step_goal'] = training_df['step_goal'].astype(float)

main_sleep preprocecssing

In [59]:
# binary encoding
training_df['main_sleep'].replace(to_replace=[True], value=[1], inplace=True)

consciousness_raising, counterconditioning, helping_relationships and stimulus_control preprocessing

In [60]:
# replace with 0,1,2 (ordinal relationship)
for feature in ['consciousness_raising_category', 'counterconditioning_category', 'helping_relationships_category', 'stimulus_control_category']:
    training_df[feature].replace(to_replace=['Below average', 'Average', 'Above average'], value=[0, 1 ,2], inplace=True)

place preprocessing

In [61]:
# one-hot-encoding
s = training_df['place']
dum = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
df = pd.concat([s, dum], axis=1)
training_df = pd.concat([training_df, df], axis=1)
training_df = training_df.drop(columns='place')

In [62]:
training_df.to_pickle('../data/basic_preprocessed_training_df.pkl')

##### 5. Preprocess the different features' granularity

In [63]:
training_df = pd.read_pickle('../data/basic_preprocessed_training_df.pkl')
training_df

,id,date,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,moderately_active_minutes,...,Treadmill,Bootcamp,ENTERTAINMENT,GYM,HOME,HOME_OFFICE,OTHER,OUTDOORS,TRANSIT,WORK/SCHOOL
164878,621e375b67b776a240290cdc,2021-01-06 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164960,621e2f6167b776a240e082a9,2021-01-06 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164856,621e301367b776a24057738e,2021-01-06 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164874,621e362467b776a2404ad513,2021-01-08 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164858,621e30c867b776a240d4aa6c,2021-01-08 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56523,621e301e67b776a240608a72,2022-01-22 00:00:00,NaN,NaN,NaN,0.9,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164964,621e310d67b776a24003096d,2022-01-24 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164870,621e346f67b776a24081744f,2022-01-24 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164855,621e300767b776a2404dc717,2022-01-24 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Group together features that need the same granularity preprocessing

In [65]:
copy_for_day = ['exertion_points', 'lightly_active_minutes', 'moderately_active_minutes', 'sedentary_minutes', 'very_active_minutes', 'step_goal', 'minutes_below_zone_1', 'minutes_in_zone_1', 'minutes_in_zone_2', 'minutes_in_zone_3', 'sleep_points', 'water_amount', 'mindfulness_goal']

In [66]:
for _, user_df in training_df.groupby(['id', 'date']):
    for feature in copy_for_day:
        non_null_feature = user_df[feature].notnull().any()
        if non_null_feature:
            training_df.loc[(training_df['id'] == user_df['id'].iloc[0]) & (training_df['date'].dt.date == user_df['date'].iloc[0].date()), feature] = user_df[feature].iloc[0]
training_df

,id,date,sleep_points,exertion_points,altitude,calories,distance,lightly_active_minutes,mindfulness_goal,moderately_active_minutes,...,Treadmill,Bootcamp,ENTERTAINMENT,GYM,HOME,HOME_OFFICE,OTHER,OUTDOORS,TRANSIT,WORK/SCHOOL
164878,621e375b67b776a240290cdc,2021-01-06 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164960,621e2f6167b776a240e082a9,2021-01-06 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164856,621e301367b776a24057738e,2021-01-06 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164874,621e362467b776a2404ad513,2021-01-08 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164858,621e30c867b776a240d4aa6c,2021-01-08 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56523,621e301e67b776a240608a72,2022-01-22 00:00:00,NaN,NaN,NaN,0.9,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164964,621e310d67b776a24003096d,2022-01-24 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164870,621e346f67b776a24081744f,2022-01-24 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164855,621e300767b776a2404dc717,2022-01-24 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
copy_for_rest_day = ['altitude', 'sleep_duration', 'minutes_to_fall_asleep', 'minutes_asleep', 'minutes_awake', 'minutes_after_wakeup', 'time_in_bed', 'sleep_efficiency', 'main_sleep']

In [ ]:
for _, user_df in training_df.groupby(['id', 'date']):
    for feature in copy_for_day:
        non_null_feature = user_df[feature].notnull().any()
        if non_null_feature:
            training_df.loc[(training_df['id'] == user_df['id'].iloc[0]) & (training_df['date'].dt.date == user_df['date'].iloc[0].date()) & (training_df['date'].dt.hour > user_df['date'].iloc[0].hour), feature] = user_df[feature].iloc[0]
training_df